In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from janelia_core.fileio import exp_reader, keller_lab
from pathlib import Path
import matplotlib.pyplot as plt
import glob
import numpy as np
import dask.array as array
import pyqtgraph as pg

In [3]:
image_folder = Path('/Volumes/Processing/SV4/CW_17-11-26/L1-561nm-HighFrequency_20171126_122633.corrected/Results/WeightFused/')
meta_data_folder = Path('/Volumes/Microscopy1/SV4/CW_17-11-26/L1-561nm-HighFrequency_20171126_122633')
#image_folder = Path('N:\SV4\CW_17-11-26\L1-561nm-HighFrequency_20171126_122633.corrected\Results\WeightFused')
#meta_data_folder = Path('O:\SV4\CW_17-11-26\L1-561nm-HighFrequency_20171126_122633')
#dest_dir = Path('E:\\bishop_work\\keller_vnc\\data')
new_image_folder = Path('E:\\bishop_work\\keller_vnc\\data\\images')
new_metadata_folder = Path('E:\\bishop_work\\keller_vnc\\data\\metadata')

In [4]:
#keller_lab.copy_exp(image_folder, meta_data_folder, dest_dir, update_int=500)

In [5]:
#exp = keller_lab.read_exp(image_folder, meta_data_folder)
exp = keller_lab.read_exp(new_image_folder, new_metadata_folder)

Searching for image files...
Creating dask.array object from 3731 image files.
Done reading in image files.


In [15]:
max_proj = array.std(exp.ts_data['imgs']['vls'][1:1000,:,:,:], 0).compute()

In [9]:
%gui qt
pg.image(max_proj[:,:,:])